In [13]:
# Dependencies
import numpy as np
import tweepy
import time
import json
import pandas as pd

# Twitter API Keys
consumer_key = "TEs2QmTsz4ylhWSqXWHOvVn99"
consumer_secret = "yA5M95uhTdeAKLondYQ9LDMPR5LWKc6wX70Yry1oRDHWtBN7gg"
access_token = "400199431-wb53H1jfbkDwY0xVS9aZbgWZrWlXcWkhXdFuBKMh"
access_token_secret = "IRvQVov2vmopxjeTutR0jn57lnMlYtr8syHQrGbLgbmW9"

# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [14]:
# Get a list of locations in and outside the belt (in-belt, out-belt)
base_data = pd.read_csv("Zipcode_States_With_Geo_Cordinates.csv")
lat = base_data.groupby("State")["Latitude"].mean()
lon = base_data.groupby("State")["Longitude"].mean()
dfff = {"lat":[], "lon":[]}
for each in lat.index:
    dfff["lat"].append(lat[each])
    dfff["lon"].append(lon[each])
hah = pd.DataFrame(dfff)
base_data_state = hah.set_index([lat.index])

base_data_state.head(2)

,lat,lon
State,,
Alabama,32.886361,-86.813639
Alaska,61.456423,-152.486981


In [15]:
base_data_state = base_data_state.reset_index()

In [ ]:
tweet_df = {}
tweet_df = pd.DataFrame(tweet_df)
tweet_df["state"] = ""
tweet_df["text"] = ""
tweet_df["lat"] = ""
tweet_df["lon"] = ""
tweet_df["followers"] = ""
tweet_df["favorites"] = ""
tweet_df["friends"] = ""
tweet_df["statuses_count"] = ""
tweet_df["compound"] = ""
tweet_df["pos"] = ""
tweet_df["neg"] = ""

tweet_df

,state,text,lat,lon,followers,favorites,friends,statuses_count,compound,pos,neg


In [ ]:
# Loop through DataFrame for each Latitude and Longitude
for index, row in base_data_state.iterrows():    
    print(row["State"])
    
    # Get Tweets for all locations
    location = "%s,%s,50mi" % (row["lat"], row["lon"])     
    public_tweets = api.search("", geocode = location, count=200)
    
    # Extract all hashtags from tweets directed to the account
    for tweet in public_tweets["statuses"]:

        # Print tweet in JSON
        # print(json.dumps(tweet, sort_keys=True, indent=4, separators=(',', ': ')))

        # Print the Tweet Text         
        print(tweet["text"])
        
        # Print tweet text
        tweet_df.set_value(index, "state", row["State"])
        tweet_df.set_value(index, "text", tweet["text"])
        tweet_df.set_value(index, "lat", row["lat"])
        tweet_df.set_value(index, "lon", row["lon"])
                              
        tweet_df.set_value(index, "followers", tweet["user"]["followers_count"])
        tweet_df.set_value(index, "favorites", tweet["user"]["favourites_count"])
        tweet_df.set_value(index, "friends", tweet["user"]["friends_count"])
        tweet_df.set_value(index, "statuses_count", tweet["user"]["statuses_count"])
 
        # Save to a csv for every state
        tweet_df.to_csv("Tweet_Dump.csv")
         

Alabama
RT @SteeleThoughts: Kids don’t usually remember lessons for a long time. But they remember kindness… and humor… and joy.
Need 1 for pcls 😊
RT @WarOnDumb: The Senate Leadership Fund just conceded to Moore, so Big Luther doesn't even get to have that honor for himself.
@CarlisleJonesAL YIKES
RT @broebong: kick it under the fridge https://t.co/B1wf0cdlti
@s7effano No one asked you
RT @YBNnahmir205: They say I should Remix The Race should I RT.  #YBN #nahmir #FreeTayK
This isn't about Trump's endorsement of Strange. Despite what Liberal media spins. It's his association with disgraced Gov. Bentley. #ALSEN
RT @ASAP_Brandino: I just aint been connecting with nobody fr.. I just ain’t been feeling shit.
RT @TuiteoPorVodka: Todo buen hombre deber saber distinguir perfectamente entre dónde meter el corazón y dónde sólo debe meter la verga.
Vibes.. https://t.co/Glg6mOTzEP
@J_Scott33 right back at ya bb 💘
Mahalo for reading! #H50 #Five0Redux https://t.co/BPvvOmswrx https://t.co/dPRB6jLT00


RT @wisedanichi: Nicholas If-Jesus-Christ-Had-Not-Died-For-Thee-Thou-Hadst-Been-Damned Barebone
That’s pre cum https://t.co/flLfjsHWAj
@RonaldKlain  https://t.co/AZZufdQex8
Me: We most likely descended from vikings.
Kids (10, 8, and 6): *now believe we are vikings*
#vikings #ancestry
RT @broebong: kick it under the fridge https://t.co/B1wf0cdlti
Alaska
【微小地震速報】
2017/09/27 5:06:45 JST, 
アメリカ合衆国 アンカレッジの西117km, 
ML1.5, TNT2.7kg, 深さ0.3km, 
https://t.co/6bQJzdLK9N
1.5 magnitude #earthquake. 74 km from Nikiski, #AK, United States https://t.co/k4kosfxPaW
USGS reports a M1.5 #earthquake 74km NW of Nikiski, Alaska on 9/26/17 @ 20:06:45 UTC https://t.co/oi83IKFfF5 #quake
【微小地震速報】
2017/09/27 2:09:21 JST, 
アメリカ合衆国 アンカレッジの西南西141km, 
ML1.7, TNT5.4kg, 深さ129.6km, 
https://t.co/pB9bgObwqA
1.9 magnitude #earthquake. 64 km from Nikiski, #AK, United States https://t.co/yjBhC9GzmD
USGS reports a M1.7 #earthquake 49km NNE of Redoubt Volcano, Alaska on 9/26/17 @ 17:09:21 UTC https://t.co/KOnIlP23py #quake
US

RT @D2REsports: Please keep kudos in your prayers, as he was in an accident and in a coma. #pray
I can feel myself changing for the better. Little by little. Day by day. :'-)
#OurSeason @dbacks ready for a home win! ⚾️❤️⚾️
nigera fucker a fucker pooper
lmao fukn flipnote studio i used to rewatch this several times over https://t.co/oeZjZpzJiK
Ya quiero estar en mi casa
Can we finish 370 tokens tonight 🤔
RT @switzsin: Sugar pine 7 is where it's at @notCIB @StevenSuptic #SP7ForStreamys  #streamys
https://t.co/mONWwgvYkk
600cc ...👀
Cook with purpose with Frontier Co-op's Organic Red Pepper Flakes and make this delicious vegetable  https://t.co/NaBGBO7YHF
@SenMikeLee Cong McCullom voting record https://t.co/LML92kivfe funded by Soros grp American Federation of Teachers… https://t.co/gz46nRqz3Y
@RunNGuns79 @voidwalkernick @TranimeGirl @Tauxelouve @SchmeckleTV @chrissyvalentyn @FawksTweets My bad
RT @lilsnail19: [drunk at a party, yelling over the music] me: DO YOU HAVE A RECYCLING BIN?
@Stl

RT @justincarrasco6: so disappointed in redwood administration and even more disappointed in @vtdnews for the poor coverage of the story in…
RT @SpookedMusic: @g_beats69 aint shit. 😂 https://t.co/e4IzOJPvJk
My Header Is My Mood 25/8
RT @evelynpastrana_: "You ain't got shit to say to me. I ain't got shit to say to you" https://t.co/qbb98qnCXE
my nephew is starting to fight, and my sister doesn’t say anything just cause he’s 2.
JUAN GABRIEL  BUENOS DIAS SEÑOR SOL escuchala en Radio Libre de Tierra Caliente, da click en el siguiente link: https://t.co/VANJiISI92
RT @FresnoBee: #BullardHigh teacher arrested in #Clovis on #rape allegations https://t.co/Oi8DIakwrm
@ZambaToapanta @JorgeGlas La verdadera revolución ciudadana son los rateros, en realidad que poca ética
RT @ZuloagaRay: @Linnners_13 Sometimes they push the ones that love them away so that way they can see if they're worth fighting for.
RT @svveetpwussy: Can rihanna make antidepressants
RT @svveetpwussy: Can rihanna make antidepre

@WhiteKeithSweat Are we getting too3ighty?
@Hart4Film Impossible to watch West Wing too many times.
Next track: Gifts From Enola - City Lights Scraped The Sky #NowPlaying #postrock
RT @TPHDenver: The 3-Part Harmony Marriage Ministry Retreat is October 13 &amp; 14. Register soon as spots will fill-up! More info: https://t.c…
RT @eddie__cheng: 推特今天改政策，将 140 的字数限制扩大一倍到 280。他们逐步推出这个放宽，大部分人目前还没能享受这个待遇。推特老板自己发了一条 280 字数的帖宣布此事。精于——或苦于——推特字数的网民便顺手给他做了批改，缩写成 139 字。 http…
A few shots be cherry rn ..
RT @joelcomm: How #BlockChain Works

#Fintech #Crypto #AI @ipfconline1 #makeyourownlane #defstar5 #ML #IoT... by #Ronvista_ https://t.co/jD…
BBC News - Dyson to make electric cars from 2020 https://t.co/yscrND7nhz
RT @indivisibleden: Register for the 10/8 Indivisible CO leadership summit &amp; learn skills for the next stage of this revolution. https://t.…
@UrFaveBinch YES YES YES
RT @psaundersdp: Great @Chuck_Nazty stat from #Rockies excellent PR:
He's just 11th player in MLB history to have at leas

...

Smart phones should let you choose lines of business in your auto correct key pad.. why does my phone keep... https://t.co/tz7gUmlXs6
#NFL NFL Team Apparel Reebok Philadelphia #Eagles Jackson 10 Football #Jersey Boys 5/6 https://t.co/5HsnxRlP4Z… https://t.co/X3ChnX2y3e
RT @YumiYoko: They all fucked each other. Every last one of em.  RT @AshGotThis: Tryna figure out these Motown family trees like https://t.…
@Its_Wrath_Hi @Fogbruiser @Dayken @TheTriggz Love ya sweetheart! ❤️
HEAD OF THE CLASS AND SHE JUST WON A SWALLOWSHIP
That's right! https://t.co/qsfA2agd3e
New tumblr post: "‘Game of Thrones’ Co-Stars Kit Harington and Rose..." https://t.co/QxA0sK8Jgx IFTTT, reddit
@mrgeekguymd Hello.  Can you send over a DM with your account info for review? -ML
A3: No. Not physical health issues. #writestuff https://t.co/Y1UaiNlGCw
Ariana Grande - Honeymoon Avenue (with lyrics) https://t.co/VH0D4gO8U7
@bomani_jones I'll take Dwayne Wade as a third scoring option behind Lebron and IT
.@MonkeyHaircut

RT @_JuGatti: When moms cook that big spaghetti pot for the whole week and you gotta start getting creative with it https://t.co/ymzAZQvFJz
RT @realDonaldTrump: 70 years ago today, the National Security Council met for the first time. Great history of advising Presidents-then &amp;…
one person followed me // automatically checked by https://t.co/5oEqEsKQOk
RT @TopherSpiro: Whoa. This is explosive. Republicans plan to RAISE the lowest tax rate and LOWER the highest tax rate. 
https://t.co/WYYHK…
@pi_arr_squared @iglvzx Reported you lonely sick fucker
RT @EdgeofSports: If you are using the memory of Pat Tillman to bash Colin Kaepernick, I say with utter assurance that Pat Tillman would ha…
@__mariangeli 🗣Freaky!Freaky! Lol
RT @PhillyD: Here goes nothing...  It's @Streamys time you Beautiful Bastards! Wanna watch? #Streamys #DeFrancoShowForStreamys https://t.co…
RT @SebGorka: TRAITOR. https://t.co/dgwqP3THNO
RT @SebGorka: 60th anniversary of Gov Orval Faubus keeping Black children out of 

Happy to finally see @DancingABC due to my work schedule! #DWTS Impressed w/ Derek's performance.
@Sneako26 @ChallengeDay Your mom is so freaking cute I swear
@chuckwoolery She doesn't like the competition.
T R I G G E R E D
RT @minitacheta: Pedí perdón. Perdoná. Asumí que extrañas. Volvé a hablar después de discutir. Escuchá. Sé gente que vale la pena. No pierd…
marry me https://t.co/X5ZOuz1bNi
@TheTylt @realDonaldTrump HE'S TOO STUPID TO REALIZE THE FACTS... HE LIVES IN AN ALTERNATIVE BUBBLE THAT HAS NO RELEVANCE TO THE REST OF US
RT @olivia_wiild: All eyez on me when I'm in https://t.co/zxtk3BpwYc  leggings🔥😋 https://t.co/HiDN3XHiCs
@LinesofLogic How about doing what God wants you to do?
RT @ModeratelyOkCos: "Wayward though my son may indeed be, he has made me proud. May you two know happiness."

📸@ZexenKnight 

#finalfantas…
RT @UrgentCatsTampa: PANDA - A1799246  3 year old male (Owner went into nursing home) w/Nala &amp; Mocha #Cats #TampaBay #Florida #AdoptMe
http…
RT @cinarte195

@CumCloserBoy EW GET AWAY
RT @tallmoonlight: Like for an indirect from mizzle is famous
•take a joke
•mutuals/non mutuals
•:)
RT @tallmoonlight: Like for an indirect from mizzle is famous
•take a joke
•mutuals/non mutuals
•:)
@NPR Thank you @NPR for Station ID interrupt over DJT diarrhea babble. Please ID yourself frequently. Dead-Air also welcome relief.
Like for an indirect from mizzle is famous
•take a joke
•mutuals/non mutuals
•:)
RT @wolvescalia: ultima cena scalia em hd passando pela sua tml rt para salvar vidas boa noite https://t.co/hU0705wDNN
Momma rena https://t.co/b8eBvsWuv8
5人から電話きてた📞
出なくてごめんなさい笑
What we not gonna do is this https://t.co/tWptISx5D3
RT @wolvescalia: ultima cena scalia em hd passando pela sua tml rt para salvar vidas boa noite https://t.co/hU0705wDNN
@seiyasg1221 せいやすげぇな。。。
RT @wolvescalia: eles nao tinham ideia que iam ficar juntos e fazer um casalzao da porra https://t.co/tMCSSChLpF
RT @wolvescalia: ultima cena scalia em hd passando pela sua tml rt para sal

@A_FarScape its 6:51 there. Sunset is at 7:15. He can fly to 7:45 with strobes
RT @billpostmus: =&gt;&gt; https://t.co/lv64oz90gp
@DarksideFloyd1 sunset at 7:17. Looks like #TuckerGott has another hour of flying max before he has to put it down… https://t.co/JIwm3OBERa
RT @1evilidiot: There's a show about politics called the intelligence report and I call bullshit.
Team girls doing Acro Yoga!!! 😍😍😍 https://t.co/R3OKEDLEvT
@BostonForTrump @duffsbuttons Establishment do nothings! Term limits the only way.
RT @TC_Biest: LIVE!

#PUBG

@TeamCarnageLLC @TwitchSharer @Retweet_Twitch @TwitchRetweetr @NightRTs @RetweetPros @mikey4545_gamer
https://t…
RT @TC_Biest: LIVE!

#PUBG

@TeamCarnageLLC @TwitchSharer @Retweet_Twitch @TwitchRetweetr @NightRTs @RetweetPros @mikey4545_gamer
https://t…
RT @wrongestwrong: https://t.co/VaxC8a8Iyg
@trnrtips Just got this Invite! So stoked for Mewtwo! https://t.co/uKYzJA8q0W
RT @wrongestwrong: https://t.co/VaxC8a8Iyg
RT @TC_Biest: LIVE!

#PUBG

@TeamCarnageLLC @

In [ ]:
# We're going to store the tweets inside a dataframe associated with the state, location, and in-belt/out-belt 

In [ ]:
# We're going to process all the tweets

In [ ]:
# We're going to group the data in-belt/out-belt